# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

<div class="span5 alert alert-info">
### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
</div>
****

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [3]:
# number of callbacks for black-sounding names
print ('The number of callbacks for black-sounding names: '+ str(sum(data[data.race=='b'].call)))

# number of callbacks for white-sounding names
print ('The number of callbacks for white-sounding names: '+ str(sum(data[data.race=='w'].call)))



The number of callbacks for black-sounding names: 157.0
The number of callbacks for white-sounding names: 235.0


In [4]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


In [5]:
n_b = sum(data.race=='b')
n_w = sum(data.race=='w')
print ('The number of black-sounding (white-sounding) resumes: '+ str(n_b) + ' ('+ str(n_w) + ')')


The number of black-sounding (white-sounding) resumes: 2435 (2435)


In [6]:
import statsmodels.stats.weightstats as ws

black = data[data.race == 'b']
white = data[data.race == 'w']
    
(t_test, p_value) = ws.ztest(black.call, white.call, value = 0, alternative='two-sided', usevar='pooled')

print ('The test statistics and p-value for the hypothesis test: the mean callbacks for black- and white-sounding names are equal ')
print("t-test = " + str(t_test) + " and p-value = "+ str(p_value))

if p_value <= 0.05:
    print ('The null hypothesis that two population mean is equal is rejected at a 0.05 level of significance.')
else:
    print ('The null hypothesis that two population mean is equal is not rejected at a 0.05 level of significance. Thus we can assume the distribution is normal')

The test statistics and p-value for the hypothesis test: the mean callbacks for black- and white-sounding names are equal 
t-test = -4.11470535675 and p-value = 3.87674291161e-05
The null hypothesis that two population mean is equal is rejected at a 0.05 level of significance.


In [7]:
stats.ttest_ind(black.call, white.call, axis=0, equal_var=True)

Ttest_indResult(statistic=-4.1147052908617514, pvalue=3.9408021031288859e-05)

In [8]:
#Theoretically the critical value should be t-distribution. 
#However, since the sample sizes are too large, it wouldn't make any difference for this example

#t_crit = stats.t.ppf( 1-(0.05/2), 2433)
#t_crit=1.960939501084251

z_crit= stats.norm.ppf( 1-(0.05/2))
pooled_var = ((n_b-1)*black.call.var() + (n_w-1)*white.call.var())/ (n_b+n_w-2)

margin_of_error = z_crit * (np.sqrt(pooled_var*((1/n_b)+(1/n_w))))

print ('The margin of error for 95% is '+str(margin_of_error))


The margin of error for 95% is 0.0152581370037


In [9]:
diff_mean = black.call.mean()-white.call.mean()
conf_int = (diff_mean-margin_of_error, diff_mean+margin_of_error )

print ('A 95% confidence interval for true population mean difference is ' + str(conf_int) )



A 95% confidence interval for true population mean difference is (-0.047290991858752573, -0.016774717851368581)


In [10]:
data.columns

Index(['id', 'ad', 'education', 'ofjobs', 'yearsexp', 'honors', 'volunteer',
       'military', 'empholes', 'occupspecific', 'occupbroad', 'workinschool',
       'email', 'computerskills', 'specialskills', 'firstname', 'sex', 'race',
       'h', 'l', 'call', 'city', 'kind', 'adid', 'fracblack', 'fracwhite',
       'lmedhhinc', 'fracdropout', 'fraccolp', 'linc', 'col', 'expminreq',
       'schoolreq', 'eoe', 'parent_sales', 'parent_emp', 'branch_sales',
       'branch_emp', 'fed', 'fracblack_empzip', 'fracwhite_empzip',
       'lmedhhinc_empzip', 'fracdropout_empzip', 'fraccolp_empzip',
       'linc_empzip', 'manager', 'supervisor', 'secretary', 'offsupport',
       'salesrep', 'retailsales', 'req', 'expreq', 'comreq', 'educreq',
       'compreq', 'orgreq', 'manuf', 'transcom', 'bankreal', 'trade',
       'busservice', 'othservice', 'missind', 'ownership'],
      dtype='object')